In [ ]:
from PIL import Image
from typing import Tuple
from easydict import EasyDict as edict

import numpy as np
from numpy.lib.function_base import percentile
import numpy.lib.recfunctions as rf
from pypcd import pypcd

from tools import plot_tools

## PCL from PCD file

In [ ]:
def pcl_from_pcd(path):
    """Loads pointclouds from pcd files

    Parameters
    ----------
        file (str): .pcd file
    Returns:
        pcl (np.ndarray): pointcloud as a numpy array of shape [n_points, m_channles]
                          n_points is calculated by lidar VFOV*HFOV
    """
    assert path[-3:] == 'pcd', 'Only "pcd" format is accepted.'
    pcd = pypcd.PointCloud.from_path(path)
    pcd = np.asarray(pcd.pc_data)
    return rf.structured_to_unstructured(pcd)


pcl_1 = pcl_from_pcd("example_data/45_lidar_c_1654855644.9875.pcd")
pcl_2 = pcl_from_pcd("example_data/45_lidar_c_1654855645.2878.pcd")

print(pcl_1.shape) # (1024 points x 64 layers, 9 channel)
print(pcl_2.shape) # (1024 points x 64 layers, 9 channel)

## BEV from PCL

In [ ]:
OUSTER_CHANNELS = ['x','y','z','intensity','t','reflectivity','ring','ambient','range']

def pcl_to_bev(pcl:np.ndarray, configs: edict) -> np.ndarray:
    """Computes the bev map of a given pointcloud. 
    
    For generality, this method can return the bev map of the available 
    channels listed in '''BEVConfig.VALID_CHANNELS'''. 

    Parameters
    ----------
        pcl (np.ndarray): pointcloud as a numpy array of shape [n_points, m_channles] 
        configs (Dict): configuration parameters of the resulting bev_map

    Returns
    -------
        bev_map (np.ndarray): bev_map as numpy array of shape [len(config.channels), configs.bev_height, configs.bev_width ]
    """
    
    # remove lidar points outside detection area and with too low reflectivity
    mask = np.where((pcl[:, 0] >= configs.lims.x[0]) & (pcl[:, 0] <= configs.lims.x[1]) &
                    (pcl[:, 1] >= configs.lims.y[0]) & (pcl[:, 1] <= configs.lims.y[1]) &
                    (pcl[:, 2] >= configs.lims.z[0]) & (pcl[:, 2] <= configs.lims.z[1]))
    pcl = pcl[mask]

    # shift level of ground plane to avoid flipping from 0 to 255 for neighboring pixels
    pcl[:, 2] = pcl[:, 2] - configs.lims.z[0]  

    # Convert sensor coordinates to bev-map coordinates (center is bottom-middle)
    # compute bev-map discretization by dividing x-range by the bev-image height
    bev_x_discret = (configs.lims.x[1] - configs.lims.x[0]) / configs.bev_height
    bev_y_discret = (configs.lims.y[1] - configs.lims.y[0]) / configs.bev_width
    ## transform all metrix x-coordinates into bev-image coordinates    
    pcl_cpy = np.copy(pcl)
    pcl_cpy[:, 0] = np.int_(np.floor(pcl_cpy[:, 0] / bev_x_discret))
    # transform all y-coordinates making sure that no negative bev-coordinates occur
    pcl_cpy[:, 1] = np.int_(np.floor(pcl_cpy[:, 1] / bev_y_discret) + (configs.bev_width + 1) / 2) 
    # Create BEV map
    bev_map = np.zeros((3, configs.bev_height, configs.bev_width))
    # Compute height and density channel
    pcl_height_sorted, counts = sort_and_map(pcl_cpy, 2, return_counts=True)
    xs = np.int_(pcl_height_sorted[:, 0])
    ys = np.int_(pcl_height_sorted[:, 1])
    # Fill height map
    normalized_height = pcl_height_sorted[:, 2]/float(np.abs(configs.lims.z[1] - configs.lims.z[0]))
    height_map = np.zeros((configs.bev_height + 1, configs.bev_width + 1))
    height_map[xs,ys] = normalized_height
    
    # Fill density map
    normalized_density = np.minimum(1.0, np.log(counts + 1) / np.log(64))
    density_map = np.zeros((configs.bev_height + 1, configs.bev_width + 1))
    density_map[xs,ys] = normalized_density

    # Compute intesity channel
    pcl_cpy[pcl_cpy[:,3]>configs.lims.intensity[1],3] = configs.lims.intensity[1]
    pcl_cpy[pcl_cpy[:,3]<configs.lims.intensity[0],3] = configs.lims.intensity[0]
    
    pcl_int_sorted, _ = sort_and_map(pcl_cpy, 3, return_counts=False)
    xs = np.int_(pcl_int_sorted[:, 0])
    ys = np.int_(pcl_int_sorted[:, 1])
    normalized_int = pcl_int_sorted[:, 3]/(np.amax(pcl_int_sorted[:, 3])-np.amin(pcl_int_sorted[:, 3]))
    intensity_map = np.zeros((configs.bev_height + 1, configs.bev_width + 1))
    intensity_map[xs,ys] = normalized_int
    
    # Fill BEV 
    bev_map[2,:,:] = density_map[:configs.bev_height, :configs.bev_width]
    bev_map[1,:,:] = height_map[:configs.bev_height, :configs.bev_width]
    bev_map[0,:,:] = intensity_map[:configs.bev_height, :configs.bev_width]
 
    return bev_map

def sort_and_map(pcl: np.ndarray, channel_index: int, return_counts:bool=False) ->Tuple[np.ndarray,np.ndarray]:
    """Function to re-arrange elements in poincloud by sorting first by x, then y, then -channel.
    This function allows users to map a pointcloud channel to a top view image (in z axis) of that channel.

    Parameters
    ----------
        pcl (np.ndarray): Input pointcloud of of shape [n_points, m_channles]
        channel_index (int): Index of channel to take into account as third factor, 
                             when sorting the pointcloud.
        return_counts (bool): True to return the counts on points per cell. Used for density channel
    Returns
     ----------
       channel_map (np.ndarray): [description]
       counts (np.ndarray): [description]
       
    """

    idx= np.lexsort((-pcl[:, channel_index], pcl[:, 1], pcl[:, 0]))
    pcl_sorted = pcl[idx]
    counts = None
    # extract all points with identical x and y such that only the maximum value of the channel is kept
    if return_counts:
        _, indices, counts = np.unique(pcl_sorted[:, 0:2], axis=0, return_index=True, return_counts=return_counts)
    else:
        _, indices = np.unique(pcl_sorted[:, 0:2], axis=0, return_index=True)
    return (pcl_sorted[indices], counts)

def show_bev_map(bev_map: np.ndarray) -> None:
    """Function to show bev_map as an RGB image

    By default, the image will only show the 3 first channels of `bev_map`. 

    Parameters
    ----------
        bev_map (np.ndarray): bev_map as numpy array of shape `[len(config.channels), configs.bev_height, configs.bev_width ]` 
    """
    bev_image: np.ndarray =  (np.swapaxes(np.swapaxes(bev_map,0,1),1,2)*255).astype(np.uint8)
    mask: np.ndarray = np.zeros_like(bev_image[:,:,0])


    height_image = Image.fromarray(np.dstack((bev_image[:, :, 0],mask,mask)))
    den_image = Image.fromarray(np.dstack((mask,bev_image[:, :, 1],mask)))
    int_image = Image.fromarray(np.dstack((mask,mask,bev_image[:, :, 2])))

    int_image.show()
    den_image.show()
    height_image.show()
    Image.fromarray(bev_image).show()


configs = edict()
configs.lims = edict()
configs.lims.x = [0, 50]
configs.lims.y = [-25, 25]
configs.lims.z = [-1, 3]
configs.lims.intensity = [0, 100.0]
configs.bev_height = 640
configs.bev_width = 640
bev_map = pcl_to_bev(pcl_2, configs)
show_bev_map(bev_map)
plot_tools.show_pcl(pcl_2)



## Range Image from PCL

In [ ]:

def pcl_to_range_image(pcl:np.ndarray, lidar_config: edict) -> np.ndarray:
    width = lidar_config.points_per_layer
    height = lidar_config.n_layers
    fov_up = lidar_config.fov_up * np.pi /180
    fov_down = lidar_config.fov_down * np.pi / 180.0

    # Calculate range and angles for each point
    ranges = np.linalg.norm(pcl[:, :3], axis=1)
    channels_info = pcl[:, 3:]
    azimuths = np.arctan2(pcl[:, 1], pcl[:, 0])
    elevations = np.arcsin(pcl[:, 2] / ranges)
    elevations[np.isnan(elevations)] = 0

    # Convert range and angles to pixel coordinates
    range_image = np.zeros((height, width), dtype=np.float32)
    intensity_image = np.zeros((height, width, pcl.shape[1]-3), dtype=np.float32)
    row_inds = np.round(((fov_down - elevations) / (fov_up - fov_down)) * (height - 1)).astype(np.int32)

    col_inds = np.round(((azimuths + np.pi) / (2*np.pi)) * (width - 1)).astype(np.int32)
    range_image[row_inds, col_inds] = ranges
    intensity_image[row_inds, col_inds] = channels_info
    range_channels_image = np.dstack((range_image, intensity_image))
    
    return range_channels_image

def show_range_image(range_image):
    ri = range_image.copy()
    deg90 = int(ri.shape[1]/4)
    ri_center = int(ri.shape[1]/2)
    ri = ri[:,ri_center-deg90:ri_center+deg90,:]
    #  map the range channel onto an 8-bit scale and make sure that the full range of values is appropriately considered
    ri_range = ri[:,:,0]
    ri_range = (ri_range-np.amin(ri_range)) * 255 / (np.amax(ri_range) - np.amin(ri_range))
    # map the intensity channel onto an 8-bit scale and normalize with the difference between the 1- and 99-percentile to mitigate the influence of outliers
    ri_intensity = ri[:,:,1]
    p99 = percentile(ri_intensity,99)
    p1  = percentile(ri_intensity,1), 
    ri_intensity = np.clip(ri_intensity, p1, p99)
    ri_intensity = 255*(ri_intensity-p1)/(p99-p1)
    # stack the range and intensity image vertically using np.vstack and convert the result to an unsigned 8-bit integer   
    img_range_intensity = np.vstack((ri_range, ri_intensity)).astype(np.uint8)
    Image.fromarray(ri_range.astype(np.uint8)).show()
    Image.fromarray(ri_intensity.astype(np.uint8)).show()
    return img_range_intensity


lidar_config = edict()
lidar_config.points_per_layer = 1024
lidar_config.n_layers = 320
lidar_config.fov_up = 62
lidar_config.fov_down = -69

ri = pcl_to_range_image(pcl_2, lidar_config)

Image.fromarray(ri[:,:,0]).show()
Image.fromarray(ri[:,:,1]).show()


## PCL from Range image

In [ ]:

def range_image_to_pointcloud(ri, lidar_config):
    # Convert FOV values from degrees to radians
    fov_up = lidar_config.fov_up * np.pi / 180.0
    fov_down = lidar_config.fov_down * np.pi / 180.0
    
    # Calculate vertical and horizontal FOV angles
    height, width, channels = ri.shape
    # Create empty point cloud array
    pcl = np.zeros((height * width, 3+(channels-1)), dtype=np.float32)
    # Calculate range and angles for each pixel
    elevations = np.linspace(fov_up, fov_down, height)
    azimuths = np.linspace(-np.pi, np.pi, width)
    azimuths, elevations = np.meshgrid(azimuths, elevations)
    ranges = ri[:,:,0]
    
    # Convert range and angles to Cartesian coordinates
    x = ranges * np.cos(elevations) * np.cos(azimuths)
    y = ranges * np.cos(elevations) * np.sin(azimuths)
    z = ranges * np.sin(elevations)
    
    # Fill in point cloud array
    pcl[:, 0] = x.flatten()
    pcl[:, 1] = y.flatten()
    pcl[:, 2] = z.flatten()

    pcl[:,3:] = np.reshape(ri[:,:,1:], (height*width,-1))
    
    # Filter out invalid points
    valid_indices = np.where(np.isfinite(pcl).all(axis=1))[0]
    pcl = pcl[valid_indices]
    
    return pcl

lidar_config = edict()
lidar_config.points_per_layer = 1024
lidar_config.n_layers = 64
lidar_config.fov_up = 22.5/2
lidar_config.fov_down = -22.5/2

pcl_reconstructed = range_image_to_pointcloud(ri,lidar_config)
plot_tools.show_pcl(pcl_reconstructed)


